<a href="https://colab.research.google.com/github/satishcse/mkt_updates/blob/main/Various_Chartink_links_download_all_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chartink results

In [ ]:
import os

folder_path = '/root/airflow/dags'

if not os.path.exists(folder_path):
  os.makedirs(folder_path)

In [ ]:
import google.auth
from googleapiclient.discovery import build
from google.colab import drive

def run_colab_notebook(notebook_id):
    # Authenticate and build the Google Colab API client
    creds, _ = google.auth.default()
    service = build('colab', 'v1', credentials=creds)

    # Mount Google Drive
    drive.mount('/content/drive', force_remount=True)

    # Open the specified notebook
    notebook = service.notebooks().get(notebookId=notebook_id).execute()
    cells = notebook.get('cells', [])

    # Execute each cell in the notebook
    for cell in cells:
        result = service.notebooks().execute(notebookId=notebook_id, body=cell).execute()


In [ ]:
from airflow.operators.python import PythonOperator
from airflow.models import DAG
from datetime import datetime, timedelta

In [ ]:
with open ('/root/airflow/dags/run_colab_notebook.py', 'w+') as f:
  f.write("default_args = {    'owner': 'airflow','depends_on_past': False,    'start_date': datetime(2023, 2, 2),    'email_on_failure': False,    'email_on_retry': False,    'retries': 1,    'retry_delay': timedelta(minutes=3)} dag = DAG(    'run_colab_notebook',    default_args=default_args,    schedule=timedelta(minutes=1)) run_colab_notebook = PythonOperator(    task_id='run_colab_notebook',    python_callable=run_colab_notebook,    dag=dag)")
  f.close()

In [ ]:
# !airflow worker

In [ ]:
# !airflow webserver -p 8000

import subprocess
subprocess.call("airflow webserver -p 8000 -k", shell=True)

2

In [ ]:
!ping 172.28.0.12
# installing ping utility
# !apt-get update && apt-get install iputils-ping -y

PING 172.28.0.12 (172.28.0.12) 56(84) bytes of data.
64 bytes from 172.28.0.12: icmp_seq=1 ttl=64 time=0.068 ms
64 bytes from 172.28.0.12: icmp_seq=2 ttl=64 time=0.060 ms
64 bytes from 172.28.0.12: icmp_seq=3 ttl=64 time=0.062 ms

--- 172.28.0.12 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2055ms
rtt min/avg/max/mdev = 0.060/0.063/0.068/0.003 ms


In [ ]:
import requests

url = "http://172.28.0.12:8000/api/experimental/dags/run_colab_notebook/dag_runs"

headers = {
    'Content-Type': 'application/json'
}

data = {
    'run_id': 'manual__' + str(datetime.now().isoformat()),
    'execution_date': str(datetime.now().isoformat())
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)

ConnectionError: ignored

In [ ]:
import socket

host_name = socket.gethostname()
host_ip = socket.gethostbyname(host_name)

print("Hostname:", host_name)
print("IP address:", host_ip)

# Hostname: 29fbdf8dee31
# IP address: 172.28.0.12

Hostname: 29fbdf8dee31
IP address: 172.28.0.12


In [ ]:
!git clone https://github.com/satishcse/mkt_updates.git
%cd mkt_updates

In [ ]:
import csv
# import sys
# import os
import datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup

Charting_Link = "https://chartink.com/screener/"
Charting_url = 'https://chartink.com/screener/process'

#You need to copy paste condition in below mentioned Condition variable

#Condition = "( {136699} ( latest high - latest low < 1 day ago high - 1 day ago low and latest high - latest low < 2 days ago high - 2 days ago low and latest high - latest low < 3 days ago high - 3 days ago low and latest high - latest low < 4 days ago high - 4 days ago low and latest high - latest low < 5 days ago high - 5 days ago low and latest high - latest low < 6 days ago high - 6 days ago low ) )"


#ID_File1:
Condition_Sector_performance = "( {45603} ( latest close > 1 ) )"

#ID_File2:
Condition_1hr_OBV_DIV = "( {33492} ( ( {33492} ( latest min( 5 , latest rsi( 14 ) ) > 5 days ago max( 5 , latest rsi( 14 ) ) and latest min( 5 , latest obv ) > 5 days ago max( 5 , latest obv ) and latest min( 5 , latest low ) < 5 days ago max( 5 , latest low ) and latest close > 1 day ago max( 3 , latest high ) ) ) ) ) "
Condition_1hr_OBV_DIV_BN = "( {136699} ( ( {136699} ( latest min( 5 , latest rsi( 14 ) ) > 5 days ago max( 5 , latest rsi( 14 ) ) and latest min( 5 , latest obv ) > 5 days ago max( 5 , latest obv ) and latest min( 5 , latest low ) < 5 days ago max( 5 , latest low ) and latest close > 1 day ago max( 3 , latest high ) ) ) ) ) "
Condition_ID_Buy_PSAR = "( {33489} ( latest parabolic sar( 0.04,0.02,0.2 ) < latest ema( close,9 ) and 1 day ago  parabolic sar( 0.04,0.02,0.2 ) >= 1 day ago  ema( close,9 ) ) )" 
Condition_ID_Volume_Shocker = "( {33489} ( ( {33489} ( [0] 5 minute wma( close,2 ) >= [0] 5 minute wma( close,3 ) and [0] 5 minute close > [-1] 5 minute high and [0] 5 minute volume >= [0] 5 minute sma( volume,10 ) * 1.250 ) ) or( {33489} ( [0] 5 minute wma( close,2 ) <= [0] 5 minute wma( close,3 ) and [0] 5 minute close < [-1] 5 minute low and [0] 5 minute volume >= [0] 5 minute sma( volume,10 ) * 1.25 ) ) ) )"
Condition_High_relative_volume_spurt = "( {33489} ( [0] 5 minute volume > ( ( [-1] 5 minute volume + [-2] 5 minute volume + [-3] 5 minute volume ) / 3 ) * 5 and latest close > 80 and latest volume > 100000 ) )"
Condition_OH = "( {cash} ( latest open = latest high and( {cash} ( ( {33489} ( latest open = latest high ) ) ) ) ) ) "
Condition_OL = "( {cash} ( latest open = latest low and( {cash} ( ( {33489} ( latest open = latest low ) ) ) ) ) ) "

#EOD_File1:
Condition_Volume_Shocker = "( {57960} ( latest volume > latest sma( volume,10 ) * 2 and( {cash} ( latest close > 1 day ago close * 1.05 or latest close < 1 day ago close * 0.95 ) ) ) )"
Condition_BN_NR7 = "( {136699} ( latest high - latest low < 1 day ago high - 1 day ago low and latest high - latest low < 2 days ago high - 2 days ago low and latest high - latest low < 3 days ago high - 3 days ago low and latest high - latest low < 4 days ago high - 4 days ago low and latest high - latest low < 5 days ago high - 5 days ago low and latest high - latest low < 6 days ago high - 6 days ago low ) ) " 
Condition_BN_NCPR = "( {136699} ( abs( ( ( ( latest high + latest low + latest close / 3 ) + ( ( latest high + latest low + latest close / 3 ) - ( latest high + latest low / 2 ) ) - ( latest high + latest low / 2 ) ) ) ) < ( latest close * 0.001 ) and latest close > 300 ) )"
Condition_NCPR_KGS = "( {33489} ( abs( ( ( ( latest high + latest low + latest close / 3 ) + ( ( latest high + latest low + latest close / 3 ) - ( latest high + latest low / 2 ) ) - ( latest high + latest low / 2 ) ) ) ) < ( latest close * 0.001 ) and latest close > 300 ) )"
Condition_OBV_RSI_Buy = "( {33489} ( latest rsi( 2 ) > 60 and( latest close - latest open ) / ( latest high - latest low ) > 0.55 and latest sma( volume,10 ) > 300000 and latest close < 1500 and latest adx di positive( 14 ) >= latest adx di negative( 14 ) and latest adx( 14 ) > 21 and latest close >= 15 ) )"
Condition_Strong_Stocks = "( {57960} ( abs( latest close - 124 days ago close ) * 100 / 124 days ago close > 40 and market cap > 500 and latest close > 1 day ago close ) )"
Condition_Bullish_Harami_N100 = "( {33619} ( 2 days ago high >= 2 days ago open and 2 days ago close <= 2 days ago open and 2 days ago low <= 2 days ago close and 1 day ago high <= 2 days ago high and 1 day ago close <= 2 days ago open and 1 day ago open >= 2 days ago close and 1 day ago low >= 2 days ago low and 1 day ago close > 1 day ago open and latest close >= 2 days ago high and( {cash} ( latest open >= 1 day ago close ) ) and latest close >= 30 ) )"


#Fundamental_scans

#FS_File1:
Condition_FII_SH_increase = "( {33489} ( quarterly foreign institutional investors percentage > 1 quarter ago foreign institutional investors percentage and quarterly foreign institutional investors percentage > 2 quarter ago foreign institutional investors percentage and weekly close > 1 week ago close and quarterly net profit/reported profit after tax > 10 and yearly return on capital employed percentage > 15 ) ) "
#Eps Increase , Stock near 15% of 52 week high , Close above 50 ema , Market cap>2000 cr, ROE and ROCE >20
Condition_Fundamental_Tenchnical_Swing = "( {cash} ( quarterly eps after extraordinary items diluted >= 1 quarter ago eps after extraordinary items diluted and 1 quarter ago eps after extraordinary items diluted >= 2 quarter ago eps after extraordinary items diluted and 2 quarter ago eps after extraordinary items diluted >= 3 quarter ago eps after extraordinary items diluted and market cap > 2000 and latest close >= latest max( 252 , latest high ) * .80 and latest close > latest ema( latest close , 50 ) and latest volume > 100000 and yearly return on capital employed percentage > 20 and yearly return on net worth percentage >= 20 ) ) "
Condition_Zero_Debt_Momentum = "( {cash} ( latest volume > 500000 and latest close > latest ema( close,200 ) and latest ema( close,50 ) > latest ema( close,200 ) and latest ema( close,50 ) > latest ema( close,100 ) and latest ema( close,40 ) > latest ema( close,80 ) and latest ema( close,20 ) > latest ema( close,50 ) and latest ema( close,150 ) > latest ema( close,250 ) and total loans = 0 and latest close > latest ema( close,150 ) and net profit[yearly] > 10 and sales turnover[yearly] > 10 ) ) "
Condition_FIPA_above_all_SMAs = "( {57960} ( latest close > latest ema( latest close , 200 ) and latest close > latest ema( latest close , 100 ) and latest close > latest ema( latest close , 50 ) and latest close > latest ema( latest close , 20 ) and latest volume > 500000 and earning per share[eps] > prev year eps and yearly net profit/reported profit after tax > 1 year ago net profit/reported profit after tax and yearly net sales > 1 year ago net sales and market cap > 10000 ) )"
Condition_Outperforming_BM_1W_3M = "( {cash} ( ( {cash} ( ( latest close - 55 days ago close ) * 100 / 55 days ago close > 12 and( latest close - 21 days ago close ) * 100 / 21 days ago close > 12 and( latest close - 123 days ago close ) * 100 / 123 days ago close > 16 and( latest close - 5 days ago close ) * 100 / 5 days ago close > 8 and latest close >= latest supertrend( 7 , 3 ) and weekly rsi( 14 ) >= 59 and monthly rsi( 14 ) >= 50 and latest rsi( 14 ) >= 50 and latest close > latest upper bollinger band( 20 , 2 ) and weekly close > latest upper bollinger band( 20 , 2 ) and monthly close > latest upper bollinger band( 20 , 2 ) and latest close > latest open and latest close > 1 day ago high ) ) and latest volume > 300000 and market cap >= 300 ) )"


#Candlestick patters scans

#CS_ID_File1:
Condition_Tweezer_top_15m_N50 = "( {33492} ( ( {cash} ( [0] 15 minute high != greatest(  [0] 15 minute open, [0] 15 minute close  ) and [-1] 15 minute high != greatest(  [-1] 15 minute open, [-1] 15 minute close  ) and [0] 15 minute high = [-1] 15 minute high ) ) or( {cash} ( [0] 15 minute high = greatest(  [0] 15 minute open, [0] 15 minute close  ) and [-1] 15 minute high = greatest(  [-1] 15 minute open, [-1] 15 minute close  ) and [0] 15 minute high = [-1] 15 minute high ) ) ) )"
Condition_Bullish_Hammer_15m_N500 = "( {57960} ( abs( [0] 15 minute open - [0] 15 minute close ) > ( [0] 15 minute high - [0] 15 minute low ) * 0.05 and abs( [0] 15 minute open - [0] 15 minute close ) < ( [0] 15 minute high - [0] 15 minute low ) * 0.50 and 0.50 * ( [0] 15 minute open + [0] 15 minute close ) - [0] 15 minute low > ( [0] 15 minute high - [0] 15 minute low ) * 0.80 and [0] 15 minute low = latest low and [0] 15 minute {custom_indicator_14779_start}\"greatest(   close,  open  ) - least(   open,  close  )\"{custom_indicator_14779_end} >= 0.80 ) )"

#CS_EOD_File1:
Condition_Hanging_Man_N100 = "( {33619} ( latest open > latest close and abs( latest open - latest close ) > ( latest high - latest low ) * 0.3 and abs( latest open - latest close ) < ( latest high - latest low ) * 0.50 and 0.50 * ( latest open + latest close ) - latest low > ( latest high - latest low * 0.70 ) and latest high = weekly high ) ) "
Condition_Bullish_Hammer_Daily_N50 = "( {33492} ( ( {33492} ( ( latest close - latest low ) > ( latest high - latest open ) * 3 and latest close >= latest open ) ) or( {33492} ( ( latest open - latest low ) > ( latest high - latest close ) * 3 and latest open >= latest close ) ) ) )"
Condition_Bullish_Hammer_weekly_N500_near_BB = "( {57960} ( latest close > 50 and latest volume > 50000 and latest rsi( 14 ) > 55 and latest adx di positive( 14 ) > latest adx di negative( 14 ) and weekly close > weekly sma( weekly close , 200 ) and( {57960} ( ( {57960} ( weekly high < weekly lower bollinger band( 20 , 2 ) * 1.03 and weekly high > weekly lower bollinger band( 20 , 2 ) * 0.975 ) ) or( {57960} ( weekly close < weekly sma( weekly close , 20 ) * 1.05 and weekly close > weekly sma( weekly close , 20 ) ) ) ) ) and( {57960} ( ( {57960} ( 1 week ago close > 1 week ago open and weekly high < 1 week ago close ) ) or( {57960} ( 1 week ago open > 1 week ago close and weekly high < 1 week ago open ) ) ) ) and( {57960} ( ( {57960} ( ( weekly high - weekly low ) > 3 * ( weekly open - weekly close ) and( weekly close - weekly low ) / ( 0.001 + weekly high - weekly low ) > 0.6 and( weekly open - weekly low ) / ( 0.001 + weekly high - weekly low ) > 0.6 ) ) or( {57960} ( weekly open < weekly close and weekly high < weekly lower bollinger band( 20 , 2 ) * 1.05 and( weekly open - weekly low ) / ( weekly high - weekly open ) >= 2 ) ) ) ) ) )"

Condition_Weekly_NCPR_KGS = "( {33489} ( abs( ( ( ( weekly high + weekly low + weekly close / 3 ) + ( ( weekly high + weekly low + weekly close / 3 ) - ( weekly high + weekly low / 2 ) ) - ( weekly high + weekly low / 2 ) ) ) ) < ( weekly close * 0.005 ) and weekly close > 100 ) )"


In [ ]:
def GetDataFromChartink(payload):
    payload = {'scan_clause': payload}

    with requests.Session() as s:
        r = s.get(Charting_Link)
        soup = BeautifulSoup(r.text, "html.parser")
        csrf = soup.select_one("[name='csrf-token']")['content']
        s.headers['x-csrf-token'] = csrf
        r = s.post(Charting_url, data=payload)

        df = pd.DataFrame()
        for item in r.json()['data']:
            df = df.append(item, ignore_index=True)
#             df = pd.concat(item)
    return df

In [ ]:
# ID_File1:
data_Sector_performance = GetDataFromChartink(Condition_Sector_performance)
if not data_Sector_performance.empty:
    data_Sector_performance.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_Sector_performance = data_Sector_performance.sort_values(by='per_chg', ascending=False)

# ID_File2:
data_1hr_OBV_DI = GetDataFromChartink(Condition_1hr_OBV_DIV)
if not data_1hr_OBV_DI.empty:
    data_1hr_OBV_DI.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_1hr_OBV_DI = data_1hr_OBV_DI.sort_values(by='per_chg', ascending=False)

data_1hr_OBV_DI_BN = GetDataFromChartink(Condition_1hr_OBV_DIV_BN)
if not data_1hr_OBV_DI_BN.empty:
    data_1hr_OBV_DI_BN.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_1hr_OBV_DI_BN = data_1hr_OBV_DI_BN.sort_values(by='per_chg', ascending=False)

data_ID_Buy_PSAR = GetDataFromChartink(Condition_ID_Buy_PSAR)
if not data_ID_Buy_PSAR.empty:
    data_ID_Buy_PSAR.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_ID_Buy_PSAR = data_ID_Buy_PSAR.sort_values(by='per_chg', ascending=False)
    
data_ID_Volume_Shocker = GetDataFromChartink(Condition_ID_Volume_Shocker)
if not data_ID_Volume_Shocker.empty:
    data_ID_Volume_Shocker.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_ID_Volume_Shocker = data_ID_Volume_Shocker.sort_values(by='volume', ascending=False)
    
data_High_relative_volume_spurt = GetDataFromChartink(Condition_High_relative_volume_spurt)
if not data_High_relative_volume_spurt.empty:
    data_High_relative_volume_spurt.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_High_relative_volume_spurt = data_High_relative_volume_spurt.sort_values(by='volume', ascending=False)

data_OH = GetDataFromChartink(Condition_OH)
if not data_OH.empty:
    data_OH.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_OH = data_OH.sort_values(by='per_chg', ascending=False)

data_OL = GetDataFromChartink(Condition_OL)
if not data_OL.empty:
    data_OL.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_OL = data_OL.sort_values(by='per_chg', ascending=False)

# CS_File1:
data_Tweezer_top_15m_N50 = GetDataFromChartink(Condition_Tweezer_top_15m_N50)
if not data_Tweezer_top_15m_N50.empty:
    data_Tweezer_top_15m_N50.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_Tweezer_top_15m_N50 = data_Tweezer_top_15m_N50.sort_values(by='volume', ascending=False)

data_Bullish_Hammer_15m_N500 = GetDataFromChartink(Condition_Bullish_Hammer_15m_N500)
if not data_Bullish_Hammer_15m_N500.empty:
    data_Bullish_Hammer_15m_N500.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
    data_Bullish_Hammer_15m_N500 = data_Bullish_Hammer_15m_N500.sort_values(by='volume', ascending=False)
    
    
if dt.datetime.now().time().strftime("%H:%M:%S") >= '15:25:00':
    # EOD_File1:    
    data_Volume_Shocker = GetDataFromChartink(Condition_Volume_Shocker)
    if not data_Volume_Shocker.empty:
        data_Volume_Shocker.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Volume_Shocker = data_Volume_Shocker.sort_values(by='per_chg', ascending=False)

    data_BN_NR7 = GetDataFromChartink(Condition_BN_NR7)
    if not data_BN_NR7.empty:
        data_BN_NR7.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_BN_NR7 = data_BN_NR7.sort_values(by='per_chg', ascending=False)

    data_BN_NCPR = GetDataFromChartink(Condition_BN_NCPR)
    if not data_BN_NCPR.empty:
        data_BN_NCPR.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_BN_NCPR = data_BN_NCPR.sort_values(by='per_chg', ascending=False)

    data_NCPR_KGS = GetDataFromChartink(Condition_NCPR_KGS)
    if not data_NCPR_KGS.empty:
        data_NCPR_KGS.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_NCPR_KGS = data_NCPR_KGS.sort_values(by='per_chg', ascending=False)
    
    data_OBV_RSI_Buy = GetDataFromChartink(Condition_OBV_RSI_Buy)
    if not data_OBV_RSI_Buy.empty:
        data_OBV_RSI_Buy.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_OBV_RSI_Buy = data_OBV_RSI_Buy.sort_values(by='per_chg', ascending=False)
    
    data_Strong_Stocks = GetDataFromChartink(Condition_Strong_Stocks)
    if not data_Strong_Stocks.empty:
        data_Strong_Stocks.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Strong_Stocks = data_Strong_Stocks.sort_values(by='per_chg', ascending=False)
    
    data_Bullish_Harami_N100 = GetDataFromChartink(Condition_Bullish_Harami_N100)
    if not data_Bullish_Harami_N100.empty:
        data_Bullish_Harami_N100.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Bullish_Harami_N100 = data_Bullish_Harami_N100.sort_values(by='per_chg', ascending=False)
    
    #FS_File1:
    data_FII_SH_increase = GetDataFromChartink(Condition_FII_SH_increase)
    if not data_FII_SH_increase.empty:
        data_FII_SH_increase.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_FII_SH_increase = data_FII_SH_increase.sort_values(by='per_chg', ascending=False)
    
    data_Fundamental_Tenchnical_Swing = GetDataFromChartink(Condition_Fundamental_Tenchnical_Swing)
    if not data_Fundamental_Tenchnical_Swing.empty:
        data_Fundamental_Tenchnical_Swing.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Fundamental_Tenchnical_Swing = data_Fundamental_Tenchnical_Swing.sort_values(by='per_chg', ascending=False)
    
    data_Zero_Debt_Momentum = GetDataFromChartink(Condition_Zero_Debt_Momentum)
    if not data_Zero_Debt_Momentum.empty:
        data_Zero_Debt_Momentum.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Zero_Debt_Momentum = data_Zero_Debt_Momentum.sort_values(by='per_chg', ascending=False)
    
    data_FIPA_above_all_SMAs = GetDataFromChartink(Condition_FIPA_above_all_SMAs)
    if not data_FIPA_above_all_SMAs.empty:
        data_FIPA_above_all_SMAs.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_FIPA_above_all_SMAs = data_FIPA_above_all_SMAs.sort_values(by='per_chg', ascending=False)
    
    data_Outperforming_BM_1W_3M = GetDataFromChartink(Condition_Outperforming_BM_1W_3M)
    if not data_Outperforming_BM_1W_3M.empty:
        data_Outperforming_BM_1W_3M.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Outperforming_BM_1W_3M = data_Outperforming_BM_1W_3M.sort_values(by='per_chg', ascending=False)
    
# CS_EOD_file1:        
    data_Hanging_Man_N100 = GetDataFromChartink(Condition_Hanging_Man_N100)
    if not data_Hanging_Man_N100.empty:
        data_Hanging_Man_N100.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Hanging_Man_N100 = data_Hanging_Man_N100.sort_values(by='volume', ascending=False)
    
    data_Bullish_Hammer_Daily_N50 = GetDataFromChartink(Condition_Bullish_Hammer_Daily_N50)
    if not data_Bullish_Hammer_Daily_N50.empty:
        data_Bullish_Hammer_Daily_N50.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Bullish_Hammer_Daily_N50 = data_Bullish_Hammer_Daily_N50.sort_values(by='volume', ascending=False)
    
    data_Bullish_Hammer_weekly_N500_near_BB = GetDataFromChartink(Condition_Bullish_Hammer_weekly_N500_near_BB)
    if not data_Bullish_Hammer_weekly_N500_near_BB.empty:
        data_Bullish_Hammer_weekly_N500_near_BB.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Bullish_Hammer_weekly_N500_near_BB = data_Bullish_Hammer_weekly_N500_near_BB.sort_values(by='volume', ascending=False)

    data_Weekly_NCPR_KGS = GetDataFromChartink(Condition_Weekly_NCPR_KGS)
    if not data_Weekly_NCPR_KGS.empty:
        data_Weekly_NCPR_KGS.drop(['name', 'bsecode','sr'], axis=1, inplace=True)
        data_Weekly_NCPR_KGS = data_Weekly_NCPR_KGS.sort_values(by='volume', ascending=False)
        
# data  = data_BN_NR7 + data_BN_NCPR
# print(data_BN_NR7)

In [ ]:
dt.datetime.now().time().strftime("%H:%M:%S")

'16:29:19'

In [ ]:
# ID_File1:
with open('Sector_Performance.csv', 'w+') as f:
    f.write('\nSector_Performance,,,, \n')    
    f.write('------------------ \n')    
data_Sector_performance.to_csv('Sector_Performance.csv', index = False, mode='a')   #, sep='\t'

# ID_File2:
# first file should be in writer mode (overwrite)
with open('Chartink_result.csv', 'w+') as f:
    f.write('\nNifty_1Hr_OBV_DIV,,,, \n')    
    f.write('----------------- \n')    
data_1hr_OBV_DI.to_csv('Chartink_result.csv', index = False, mode='a')

with open('Chartink_result.csv', 'a') as f:
    f.write('\nBN_1Hr_OBV_DIV,,,, \n')    
    f.write('-------------- \n')    
data_1hr_OBV_DI_BN.to_csv('Chartink_result.csv', index = False, mode='a')

with open('Chartink_result.csv', 'a') as f:
    f.write('\nIntraday_Buy_PSAR,,,, \n')    
    f.write('----------------- \n')    
data_ID_Buy_PSAR.to_csv('Chartink_result.csv', index = False, mode='a')

with open('Chartink_result.csv', 'a') as f:
    f.write('\nIntraday_Volume_Shocker,,,, \n')    
    f.write('----------------------- \n')    
data_ID_Volume_Shocker.to_csv('Chartink_result.csv', index = False, mode='a')

with open('Chartink_result.csv', 'a') as f:
    f.write('\nHigh_Relative_Volume_Spurt_5m,,,, \n')    
    f.write('----------------------------- \n')    
data_High_relative_volume_spurt.to_csv('Chartink_result.csv', index = False, mode='a')

with open('Chartink_result.csv', 'a') as f:
    f.write('\nOpen_High,,,, \n')    
    f.write('--------- \n')    
data_OH.to_csv('Chartink_result.csv', index = False, mode='a')

with open('Chartink_result.csv', 'a') as f:
    f.write('\nOpen_Low,,,, \n')    
    f.write('-------- \n')    
data_OL.to_csv('Chartink_result.csv', index = False, mode='a')

# first file should be in writer mode (overwrite)
with open('Candlesticks_Scans_ID.csv', 'w+') as f:
    f.write('\nTweezer_top_15m_Nifty,,,, \n')    
    f.write('--------------------- \n')    
data_Tweezer_top_15m_N50.to_csv('Candlesticks_Scans_ID.csv', index = False, mode='a')

with open('Candlesticks_Scans_ID.csv', 'a') as f:
    f.write('\nBullish_Hammer_15m_N500,,,, \n')    
    f.write('----------------------- \n')    
data_Bullish_Hammer_15m_N500.to_csv('Candlesticks_Scans_ID.csv', index = False, mode='a')


if dt.datetime.now().time().strftime("%H:%M:%S") >= '15:25:00':
# EOD_File1:
    with open('EOD_Checks.csv', 'w+') as f:
        f.write('\nVolume_Shocker_Daily,,,, \n')    
        f.write('-------------------- \n')    
    data_Volume_Shocker.to_csv('EOD_Checks.csv', index = False, mode='a')

    with open('EOD_Checks.csv', 'a') as f:
        f.write('\nBN_NR7,,,, \n')    
        f.write('------ \n')    
    data_BN_NR7.to_csv("EOD_Checks.csv", index=False, mode='a')

    with open('EOD_Checks.csv', 'a') as f:
        f.write('\nBN_NCPR,,,, \n')    
        f.write('------- \n')    
    data_BN_NCPR.to_csv('EOD_Checks.csv', index = False, mode='a')

    with open('EOD_Checks.csv', 'a') as f:
        f.write('\nNCPR_KGS,,,, \n')    
        f.write('-------- \n')    
    data_NCPR_KGS.to_csv('EOD_Checks.csv', index = False, mode='a')

    with open('EOD_Checks.csv', 'a') as f:
        f.write('\nOBV_RSI_Buy_Daily,,,, \n')    
        f.write('----------------- \n')    
    data_OBV_RSI_Buy.to_csv('EOD_Checks.csv', index = False, mode='a')

    with open('EOD_Checks.csv', 'a') as f:
        f.write('\nStrong_Stocks_Daily,,,, \n')    
        f.write('------------------- \n')    
    data_Strong_Stocks.to_csv('EOD_Checks.csv', index = False, mode='a')

    with open('EOD_Checks.csv', 'a') as f:
        f.write('\nBullish_Harami_N100,,,, \n')    
        f.write('------------------- \n')    
    data_Bullish_Harami_N100.to_csv('EOD_Checks.csv', index = False, mode='a')

# FS_File1:
    with open('Fundamental_Checks.csv', 'w+') as f:
        f.write('\nFII_Shareholding_Increased,,,, \n')    
        f.write('-------------------------- \n')    
    data_FII_SH_increase.to_csv('Fundamental_Checks.csv', index = False, mode='a')

    with open('Fundamental_Checks.csv', 'a') as f:
        f.write('\nFundamental_Tenchnical_Swing,,,, \n')    
        f.write('---------------------------- \n')    
    data_Fundamental_Tenchnical_Swing.to_csv('Fundamental_Checks.csv', index = False, mode='a')

    with open('Fundamental_Checks.csv', 'a') as f:
        f.write('\nZero_Debt_Momentum,,,, \n')    
        f.write('------------------ \n')    
    data_Zero_Debt_Momentum.to_csv('Fundamental_Checks.csv', index = False, mode='a')

    with open('Fundamental_Checks.csv', 'a') as f:
        f.write('\nFIPA_above_all_SMAs,,,, \n')    
        f.write('------------------- \n')    
    data_FIPA_above_all_SMAs.to_csv('Fundamental_Checks.csv', index = False, mode='a')

    with open('Fundamental_Checks.csv', 'a') as f:
        f.write('\nOutperforming_Nifty_1W_3M,,,, \n')    
        f.write('------------------------- \n')    
    data_Outperforming_BM_1W_3M.to_csv('Fundamental_Checks.csv', index = False, mode='a')

# CS_EOD_File1:
    with open('Candlestick_EOD_Scans.csv', 'w+') as f:
        f.write('\nHanging_Man_N100,,,, \n')    
        f.write('---------------- \n')    
    data_Hanging_Man_N100.to_csv('Candlestick_EOD_Scans.csv', index = False, mode='a')

    with open('Candlestick_EOD_Scans.csv', 'a') as f:
        f.write('\nBullish_Hammer_Daily_N50,,,, \n')    
        f.write('------------------------ \n')    
    data_Bullish_Hammer_Daily_N50.to_csv('Candlestick_EOD_Scans.csv', index = False, mode='a')

    with open('Candlestick_EOD_Scans.csv', 'a') as f:
        f.write('\nBullish_Hammer_weekly_near_BB_N500,,,, \n')    
        f.write('---------------------------------- \n')    
    data_Bullish_Hammer_weekly_N500_near_BB.to_csv('Candlestick_EOD_Scans.csv', index = False, mode='a')

# adding weekly NCPR here to check with the weekly candlestick scans

    with open('Candlestick_EOD_Scans.csv', 'a') as f:
        f.write('\nWekly_NCPR_KGS,,,, \n')    
        f.write('-------------- \n')    
    data_Weekly_NCPR_KGS.to_csv('Candlestick_EOD_Scans.csv', index = False, mode='a')


# Convert comma-delimited CSV files to pipe-delimited files
# Usage: Drag-and-drop CSV file over script to convert it.

# inputPath = sys.argv[1]
# outputPath = os.path.dirname(inputPath) + "/output.csv"

# https://stackoverflow.com/a/52410395/3357935
# print("Converting CSV to tab-delimited file...")
# with open('Chartink_result.csv') as inputFile:
    # newline='' prevents extra newlines when using Python 3 on Windows
    # https://stackoverflow.com/a/3348664/3357935
#     with open('Chartink_result_final.csv', 'w', newline='\r\n') as outputFile:
#         reader = csv.DictReader(inputFile, delimiter=',')
#         writer = csv.DictWriter(outputFile, reader.fieldnames, delimiter='\t')
#         writer.writeheader()
#         writer.writerows(reader)
# print("Conversion complete.")
# data_BN_NCPR.to_csv("Chartink_result.csv", mode='a', index=False, header=False, lineterminator='\r\n')
f.close()

In [ ]:
# reader = csv.reader(open("Chartink_result.csv", "r"), delimiter=',')
# writer = csv.writer(open("Chartink_result_final.csv", 'w', newline=''), delimiter='\t')
# writer.writerows(reader)
import os.path
if os.path.exists('Chartink_result.csv'):
  print ('yes')
else:
  print ('no')

yes


In [ ]:
if os.path.exists('/content/mkt_updates/Chartink_result.csv'):
  with open('/content/mkt_updates/Chartink_result.csv', mode='r') as fin, open('/content/mkt_updates/Chartink_result_Final.txt', 'w') as fout:
      for line in fin:
          fout.write(line.replace(',', '\t\t'))
  fin.close()
  fout.close()

if os.path.exists('/content/mkt_updates/Sector_Performance.csv'):
  with open('/content/mkt_updates/Sector_Performance.csv', mode='r') as fin, open('/content/mkt_updates/Sector_Performance_Final.txt', 'w') as fout:
      for line in fin:
          fout.write(line.replace(',', '\t\t'))
  fin.close()
  fout.close()

if os.path.exists('/content/mkt_updates/Candlesticks_Scans_ID.csv'):
  with open('/content/mkt_updates/Candlesticks_Scans_ID.csv', mode='r') as fin, open('/content/mkt_updates/Candlesticks_Scans_ID_Final.txt', 'w') as fout:
      for line in fin:
          fout.write(line.replace(',', '\t\t'))
  fin.close()
  fout.close()

if os.path.exists('/content/mkt_updates/EOD_Checks.csv'):
  with open('/content/mkt_updates/EOD_Checks.csv', mode='r') as fin, open('/content/mkt_updates/EOD_Checks_Final.txt', 'w') as fout:
      for line in fin:
          fout.write(line.replace(',', '\t\t'))
  fin.close()
  fout.close()

if os.path.exists('/content/mkt_updates/Fundamental_Checks.csv'):
  with open('/content/mkt_updates/Fundamental_Checks.csv', mode='r') as fin, open('/content/mkt_updates/Fundamental_Checks_Final.txt', 'w') as fout:
      for line in fin:
          fout.write(line.replace(',', '\t\t'))
  fin.close()
  fout.close()

if os.path.exists('/content/mkt_updates/Candlestick_EOD_Scans.csv'):
  with open('/content/mkt_updates/Candlestick_EOD_Scans.csv', mode='r') as fin, open('/content/mkt_updates/Candlestick_EOD_Scans_Final.txt', 'w') as fout:
      for line in fin:
          fout.write(line.replace(',', '\t\t'))
  fin.close()
  fout.close()

In [ ]:
# f = open('Chartink_result_final.csv','r')
fin.close()
fout.close()


In [ ]:
# !git config --global user.email 'satishcse2004@gmail.com'

In [ ]:
!git add /content/mkt_updates/*.csv
!git add /content/mkt_updates/*.txt
!git commit -m "updating files from Colab"
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
# !pip install git+https://github.com/bitnom/colabctl
!pip install jedi==0.10
!pip uninstall pytest
!pip install pluggy==1.0
!pip uninsytall notebook
!pip install jinja2==3.0.0
!pip install apache-airflow

In [ ]:
# !git clone https://github.com/bitnom/colabctl /content/colabctl

In [ ]:
# import csv
# with open('/content/colabctl/notebooks.csv', 'r') as f:
#   reader = csv.reader(f)
#   for row in reader:
#     print (row)

['https://colab.research.google.com/drive/1OhRD482XDuBACNStNXZ6JWjkBjPpK6yp#scrollTo=1fd8c302']


In [ ]:
# with open('/content/colabctl/notebooks.csv', 'w') as f:
#   # content = f.read()
#   # content = content.replace('old', 'new') # to change specific text
#   f.write('https://colab.research.google.com/drive/1OhRD482XDuBACNStNXZ6JWjkBjPpK6yp#scrollTo=1fd8c302')

# f.close()

In [ ]:
print("forkin"+"me")

forkinme


**Creating function to run the notebook**

---



In [ ]:
# to mount the drive inside google colab
# from google.colab import drive
# drive.mount('/content/drive')